# Lecture 6. Q Network

Q-Table은 Large Number에서 쓸수 없게 된다.

pow(2, 80\*80)과 같은 경우

그래서 Q-function Approximation을 사용한다.
Q-function Approximation에서는 뉴럴 네트워크를 사용하고 그 값으로 업데이트를 한다.

H(x) = Wx  
cost(W) = (Ws - y)^2  

hat = approximation  
theta = weight or network  
pi = 전처리

## Algorithm

__Deep Q-learning__

Initialize action-value function Q with random weights  
for episode = 1, M do  
    Initialise sequence s_1 = {x_1} and preprocessed sequenced pi_1 = pi(s_1)  
    for t = 1, *T* do  
        With probability epsilon select a random action a_t  
        otherwise select a_t = max_a Q*(pi(s_t), a; theta)  
        Execute action a_t in emulator and obseve reward gamma_t and image x_t+1  
        Set s_t+1 = s_t, a_t, x_t+1 and preprocess pi_t+1 = pi(s_t+1)  
        
        Set y_j = gamma_j for terminal pi_j+1  
                  gamma_j + gamma*\max_a'Q(pi_j+1, a';theta) for non-terminal pi_j+1  
        Perform a gradient descent step on (y_j - Q(pi_j,a_j; theta))^2 according to equation 3  
     end for
end for

DQN : Deep, Replay, Separated networks

In [ ]:
import numpy as np

In [ ]:
print(np.identity(16)[0:1])

In [ ]:
print(np.eye(16)[10:11])

In [ ]:
print(np.identity(16))

In [ ]:
def one_hot(x):
    return np.identity(16)[x:x + 1]

In [ ]:
import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
env = gym.make('FrozenLake-v0')

input_size = env.observation_space.n
output_size = env.action_space.n
learning_rate = 0.1

X = tf.placeholder(shape=[1, input_size], dtype=tf.float32)
W = tf.Variable(tf.random_uniform([input_size, output_size], 0, 0.01))

Qpred = tf.matmul(X, W)
Y = tf.placeholder(shape=[1, output_size], dtype=tf.float32)

loss = tf.reduce_sum(tf.square(Y - Qpred))

train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

dis = 0.99
num_episodes = 2000

rList = []

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(num_episodes):
        s = env.reset()
        e = 1 / ((i / 50) + 10)
        rAll = 0
        done = False
        local_loss = []
        
        while not done:
            Qs = sess.run(Qpred, feed_dict={X: one_hot(s)})
            if np.random.rand(1) < e:
                a = env.action_space.sample()
            else:
                a = np.argmax(Qs)
                
            s1, reward, done, _ = env.step(a)
            if done:
                Qs[0, a] = reward
            else:
                # Obtain the Q_s1 values by feeding the new state through our network
                Qs1 = sess.run(Qpred, feed_dict= {X: one_hot(s1)})
                # Update Q
                Qs[0, a] = reward + dis * np.max(Qs1)
                
            sess.run(train, feed_dict={X: one_hot(s), Y: Qs})
            
            rAll += reward
            s = s1
        rList.append(rAll)

In [ ]:
print("Percent of successful episodes: " + str(sum(rList)/num_episodes) + "%")
plt.bar(range(len(rList)), rList, color="blue")
plt.show()

- 학습 속도를 올리고 싶은데 어떻게 해야 하나? minibatch
- Stable 하지 않은데 어떻게 해야 하나?

Cart Pole

In [ ]:
import gym

env = gym.make('CartPole-v0')
env.reset()
random_episodes = 0
reward_sum = 0

while random_episodes < 10:
    env.render()
    action = env.action_space.sample()
    observation, reward, done, _ = env.step(action)
    print(observation, reward, done)
    reward_sum += reward
    if done:
        random_episodes += 1
        print("Reward for this episode was:", reward_sum)
        reward_sum = 0
        env.reset()
        

In [1]:
import numpy as np
import tensorflow as tf

import gym

/Users/taejunhwang/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
env = gym.make('CartPole-v0')


In [6]:
learning_rate = 1e-1
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

X = tf.placeholder(tf.float32, [None, input_size], name="input_x")

W1 = tf.get_variable("W1", shape=[input_size, output_size], 
                    initializer=tf.contrib.layers.xavier_initializer())
Qpred = tf.matmul(X, W1)

Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)

loss = tf.reduce_sum(tf.square(Y - Qpred))

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


num_episode = 5000
dis = 0.9
rList = []

# Setting up out environment
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(num_episode):
    e = 1 / ((i / 10) + 1)
    step_count = 0
    s = env.reset()
    done = False
    
    while not done :
        step_count += 1
        # 우리가 입력하는 방식에 맞게 reshape 해주는 것.
        x = np.reshape(s, [1, input_size])
        
        # predict한 값을 받아 오는것
        Qs = sess.run(Qpred, feed_dict={X: x})
        if np.random.rand(1) < e:
            a = env.action_space.sample()
        else:
            a = np.argmax(Qs)
        
        # Get new state and reward from environment
        s1, reward, done, _ = env.step(a)
        if done:
            Qs[0, a] = -100
        else:
            x1 = np.reshape(s1, [1, input_size])
            
            Qs1 = sess.run(Qpred, feed_dict={X: x1})
            # Update 시켜 줌 Q target
            Qs[0, a] = reward + dis * np.max(Qs1)
            
        # Train our network using target and predicted 
        sess.run(train, feed_dict={X: x, Y: Qs})
        s = s1
    
    rList.append(step_count)
    print("Episode: {} steps: {}".format(i, step_count))
    
    # If last 10s avg steps are 500, it's good enough
    if len(rList) > 10 and np.mean(rList[-10:]) > 500:
        braek
        

    

ValueError: Variable W1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-5-52ade94067b7>", line 8, in <module>
    initializer=tf.contrib.layers.xavier_initializer())
  File "/Users/taejunhwang/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/taejunhwang/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):


In [ ]:
observation = env.reset()
reward_sum = 0
while True:
    env.render()
    
    x = np.reshape(observation, [1, input_size])
    Qs = sess.run(Qpred, feed_dict={X: x})
    a = np.argmax(Qs)
    
    observation, reward, done, _ = env.step(a)
    reward_sum += reward
    
    if done:
        print("Total score: {}".format(reward_sum))
        break